In [1]:
# load required packages
import pymysql
import decouple
import requests
import json
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings
from datetime import datetime as dt

warnings.filterwarnings("ignore")

In [2]:
# check db connection
config = decouple.AutoConfig(' ')

user = config('AM_AWS_MY_DB_ROUSER_USER')
host = config('AM_AWS_MY_DB_ROUSER_HOST')
dbname = config('AM_AWS_MY_DB_ROUSER_DBNAME')
pw = config('AM_AWS_MY_DB_ROUSER_PW')
port = int(config('AM_AWS_MY_DB_ROUSER_PORT'))

con = pymysql.connect(host=host,
                      user=user, 
                      database=dbname,
                      password=pw,
                      port=port
                     )
cur = con.cursor()

cur.execute('SHOW TABLES')
for table_name in cur:
    print(table_name)

con.close()

if (con.open != True):
    print("Connection is closed")

('UFC_dateFight',)
('google_elev_68sites',)
('imdb_boxOffice',)
('imdb_details',)
('imdb_details_extd',)
('imdb_top250_movies',)
('imdb_top250_shows',)
Connection is closed


In [4]:
# load data from andrew db and write to csv
con = pymysql.connect(host=host,
                      user=user, 
                      database=dbname,
                      password=pw,
                      port=port
                     )
cur = con.cursor()

qstr = "SELECT * FROM imdb_details_extd"
imdb_details = pd.read_sql(qstr, con)

f =  '/Users/dylanjorling/UCLA/418/final_project/data/imdb_details.csv'
imdb_details.to_csv(f)

con.close()

In [3]:
######skip to here to directly load csv from local drive###
f =  '/Users/dylanjorling/UCLA/418/final_project/data/imdb_details.csv'
imdb_details = pd.read_csv(f)
imdb_details = imdb_details.drop('Unnamed: 0', axis=1) # for some reason i had this random extra column
imdb_mv = imdb_details[imdb_details['type'] == 'Movie'].reset_index().drop('index', axis=1).set_index('id')
imdb_mv.shape

(5098, 22)

In [4]:
imdb_mv.head()

,title,fullTitle,type,year,date,runtime,plot,awards,directors,writers,...,companies,languages,rating,imDbRatingCount,imDbRating,metacriticRating,budget,grossUSA,grossWorldwide,keywords
id,,,,,,,,,,,,,,,,,,,,,
tt0010323,The Cabinet of Dr. Caligari,The Cabinet of Dr. Caligari (1920),Movie,1920,19200227.0,67.0,"Francis, a young man, recalls in his memory th...","Awards, 1 win & 1 nomination",Robert Wiene,"Carl Mayer, Hans Janowitz",...,Decla-Bioscop AG,German,Not Rated,62574,8.0,NaN,18000.0,8811.0,8811.0,"somnambulist,mental patient,sleepwalking,mind ..."
tt0012349,The Kid,The Kid (1921),Movie,1921,19210206.0,68.0,The opening title reads: A comedy with a smile...,"Top rated movie #129 | Awards, 2 wins",Charles Chaplin,Charles Chaplin,...,Charles Chaplin Productions,English,Passed,124417,8.3,NaN,250000.0,NaN,41960.0,"orphan,the little tramp character,baby,boy,1910s"
tt0013442,Nosferatu,Nosferatu (1922),Movie,1922,19220518.0,94.0,"Wisbourg, Germany based estate agent Knock dis...","Awards, 1 win & 2 nominations",F.W. Murnau,"Henrik Galeen, Bram Stoker",...,"Jofa-Atelier Berlin-Johannisthal, Prana-Film GmbH","German, English",Not Rated,95262,7.9,NaN,NaN,NaN,19054.0,"vampire,gothic,expressionism,monster,silent film"
tt0015324,Sherlock Jr.,Sherlock Jr. (1924),Movie,1924,19240511.0,NaN,"A meek and mild projectionist, who also cleans...","Top rated movie #192 | Awards, 1 win",Buster Keaton,"Jean C. Havez, Joseph A. Mitchell, Clyde Bruckman",...,Buster Keaton Productions,"English, None",Passed,48380,8.2,NaN,NaN,977375.0,NaN,"false accusation,surrealism,national film regi..."
tt0015648,Battleship Potemkin,Battleship Potemkin (1925),Movie,1925,19251224.0,66.0,Based on the historical events the movie tells...,"Awards, 1 win",Sergei Eisenstein,"Nina Agadzhanova, Sergei Eisenstein, Grigoriy ...",...,Mosfilm,"Russian, English",Not Rated,57012,8.0,97.0,NaN,51198.0,61389.0,"imperial russian navy,1905 russian revolution,..."


In [5]:
# extract oscars won
imdb_mv['awards'].fillna('0', inplace=True)
oscars_won = []
for text in imdb_mv['awards']:
    
    x = re.search("Won (\\d+) Oscar", text)
    if x == None:
        oscars_won.append(0)
    else:
        y = re.findall(r'\d+', x[0])
        oscars_won.append(int(y[0]))
imdb_mv['oscars_won'] = oscars_won

In [6]:
# extract oscars nom
oscars_nom = []
for i, text in enumerate(imdb_mv['awards']):
    onom = imdb_mv['oscars_won'][i]
    x = re.search("Nominated for (\\d+) Oscar", text)
    if x == None:
        oscars_nom.append(onom)
    else:
        y = re.findall(r'\d+', x[0])
        onom += int(y[0])
        oscars_nom.append(onom)
imdb_mv['oscars_nom'] = oscars_nom

In [7]:
# extract award wins
award_wins = []
for i, text in enumerate(imdb_mv['awards']):
    onom = 0
    x1 = re.search("Awards, (\\d+) win", text)
    if x1 != None:
        y = re.findall(r'\d+', x1[0])
        onom += int(y[0])
    elif re.search("(\\d+) win", text) != None:
        y = re.findall(r'\d+', re.search("(\\d+) win", text)[0])
        onom += int(y[0])
    award_wins.append(onom)
imdb_mv['award_wins'] = award_wins

In [8]:
# extract award noms
award_noms = []
for i, text in enumerate(imdb_mv['awards']):
    onom = imdb_mv['award_wins'][i]
    x = re.search("(\\d+) nomination", text)
    if x != None:
        y = re.findall(r'\d+', x[0])
        onom += int(y[0])
    award_noms.append(onom)
imdb_mv['award_noms'] = award_noms  

In [9]:
# create lists and get frequencies
imdb_mv.dropna(subset=['directors', 'writers', 'stars', 'companies'], inplace=True) # net around 50 nas for these cats

director_ls = []
for d in imdb_mv['directors']:
    director_ls.append([dr for dr in d.split(sep=',')])

writer_ls = []
for w in imdb_mv['writers']:
    writer_ls.append([wr for wr in w.split(sep=',')])

stars_ls = []
for s in imdb_mv['stars']:
    stars_ls.append([sr for sr in s.split(sep=',')])
    
companies_ls = []
for c in imdb_mv['companies']:
    companies_ls.append([co for co in c.split(sep=',')])

xd = [i for sublist in director_ls for i in sublist]
xd = pd.Series(xd)
d_tab = xd.value_counts()

xw = [i for sublist in writer_ls for i in sublist]
xw = pd.Series(xw)
w_tab = xw.value_counts()

xs = [i for sublist in stars_ls for i in sublist]
xs = pd.Series(xs)
s_tab = xs.value_counts()

xc = [i for sublist in companies_ls for i in sublist]
xc = pd.Series(xc)
c_tab = xc.value_counts()

In [10]:
# create dir/writer pop, starpower, and companies
dir_pop = []
for ds in director_ls:
    dir_ls = []
    for d in ds:
        dir_count = d_tab.loc[d]
        dir_ls.append(dir_count)
    dir_pop.append(max(dir_ls)) # since usually associate movie with 1 director, took the highest freq one
imdb_mv['dir_pop'] = dir_pop

wr_pop = []
for wr in writer_ls:
    wr_ls = []
    for w in wr:
        wr_count = w_tab.loc[w]
        wr_ls.append(wr_count)
    wr_pop.append(np.mean(wr_ls)) # take mean for writers
imdb_mv['wr_pop'] = wr_pop

star_power = []
for sr in stars_ls:
    sr_ls = []
    for s in sr:
        sr_count = s_tab.loc[s]
        sr_ls.append(sr_count)
    star_power.append(sum(sr_ls)) # take mean for writers
imdb_mv['star_power'] = star_power

co_size = []
for co in companies_ls:
    co_ls = []
    for c in co:
        co_count = c_tab.loc[c]
        co_ls.append(co_count)
    co_size.append(max(co_ls)) # since usually associate movie with 1 director, took the highest freq one
imdb_mv['company_size'] = co_size


In [28]:
dir_pop_dict = {}
for ds in director_ls:
    for d in ds:
        d = d.rstrip()
        d = d.lstrip()
        if d in dir_pop_dict:
            dir_pop_dict[d] += 1
        else:
            dir_pop_dict[d] = 1

wr_pop_dict = {}
for wr in writer_ls:
    for w in wr:
        w = w.rstrip()
        w = w.lstrip()
        if w in wr_pop_dict:
            wr_pop_dict[w] += 1
        else:
            wr_pop_dict[w] = 1


star_power_dict = {}
for sr in stars_ls:
    for s in sr:
        s = s.rstrip()
        s = s.lstrip()
        if s in star_power_dict:
            star_power_dict[s] += 1
        else:
            star_power_dict[s] = 1

input_dicts = {'dir': dir_pop_dict,
              'wr': wr_pop_dict,
              'st': star_power_dict}

In [29]:
with open("input_dict.json", "w") as outfile:
    json.dump(input_dicts, outfile)


In [15]:
imdb_mv.head()

,title,fullTitle,type,year,date,runtime,plot,awards,directors,writers,...,grossWorldwide,keywords,oscars_won,oscars_nom,award_wins,award_noms,dir_pop,wr_pop,star_power,company_size
id,,,,,,,,,,,,,,,,,,,,,
tt0010323,The Cabinet of Dr. Caligari,The Cabinet of Dr. Caligari (1920),Movie,1920,19200227.0,67.0,"Francis, a young man, recalls in his memory th...","Awards, 1 win & 1 nomination",Robert Wiene,"Carl Mayer, Hans Janowitz",...,8811.0,"somnambulist,mental patient,sleepwalking,mind ...",0,0,1,2,1,1.000000,3,1
tt0012349,The Kid,The Kid (1921),Movie,1921,19210206.0,68.0,The opening title reads: A comedy with a smile...,"Top rated movie #129 | Awards, 2 wins",Charles Chaplin,Charles Chaplin,...,41960.0,"orphan,the little tramp character,baby,boy,1910s",0,0,2,2,6,6.000000,8,6
tt0013442,Nosferatu,Nosferatu (1922),Movie,1922,19220518.0,94.0,"Wisbourg, Germany based estate agent Knock dis...","Awards, 1 win & 2 nominations",F.W. Murnau,"Henrik Galeen, Bram Stoker",...,19054.0,"vampire,gothic,expressionism,monster,silent film",0,0,1,3,1,2.000000,3,1
tt0015324,Sherlock Jr.,Sherlock Jr. (1924),Movie,1924,19240511.0,NaN,"A meek and mild projectionist, who also cleans...","Top rated movie #192 | Awards, 1 win",Buster Keaton,"Jean C. Havez, Joseph A. Mitchell, Clyde Bruckman",...,NaN,"false accusation,surrealism,national film regi...",0,0,1,1,1,1.333333,4,2
tt0015648,Battleship Potemkin,Battleship Potemkin (1925),Movie,1925,19251224.0,66.0,Based on the historical events the movie tells...,"Awards, 1 win",Sergei Eisenstein,"Nina Agadzhanova, Sergei Eisenstein, Grigoriy ...",...,61389.0,"imperial russian navy,1905 russian revolution,...",0,0,1,1,1,1.000000,3,6


In [462]:
# clean date and extract month
imdb_mv['date'].isna().sum() # there are only 38 missing dates so will drop these
imdb_mv.dropna(subset=['date'], inplace=True)
imdb_mv['date'] = pd.to_datetime(imdb_mv['date'], format='%Y%m%d')
imdb_mv['release_month'] = imdb_mv['date'].dt.month_name()
imdb_mv['release_month'].head()

id
tt0010323    February
tt0012349    February
tt0013442         May
tt0015324         May
tt0015648    December
Name: release_month, dtype: object

In [463]:
# load inflation data 
inflation = pd.read_csv('/Users/dylanjorling/UCLA/418/final_project/data/inflation.csv')
inflation = inflation[['YEAR', 'AVE']]
inflation = inflation[:109]
inflation['YEAR'] = inflation['YEAR'].astype(int)
inflation['AVE'] = inflation['AVE'] / 100
inflation.set_index('YEAR', inplace=True)
inflation.sort_index(ascending=False, inplace=True)
inflation.head()

,AVE
YEAR,
2022,0.080
2021,0.047
2020,0.012
2019,0.018
2018,0.024


In [464]:
# create inflation index and convert budget column to 2023 dollars
inflation_index = []
value = 1
for val in inflation['AVE']:
    value *= (1 + val)
    inflation_index.append(value)
inflation['inflation_index'] = inflation_index

imdb_ii = []
for year in imdb_mv['date'].dt.year:
    ii = inflation.loc[year]['inflation_index']
    imdb_ii.append(ii)
imdb_mv['inflation_index'] = imdb_ii
imdb_mv['adj_budget'] = imdb_mv['budget'] * imdb_mv['inflation_index'] # note there are 697 nas

In [470]:
# save csv....this was data used for oscar analysis...contains nas in many of the financial columns still'
imdb_mv.to_csv('/Users/dylanjorling/UCLA/418/final_project/data/imdb_movies_oscar_analysis.csv')